## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-15-18-53-12 +0000,nyt,Live Updates: Trump Lands in Alaska for Summit...,https://www.nytimes.com/live/2025/08/15/world/...
1,2025-08-15-18-47-43 +0000,nyt,Trump Is Testing D.C.’s Home Rule. What Is It?,https://www.nytimes.com/2025/08/15/upshot/wash...
2,2025-08-15-18-46-35 +0000,nypost,Three babies die amid ‘concerning rise’ of syp...,https://nypost.com/2025/08/15/health/3-babies-...
3,2025-08-15-18-35-15 +0000,nyt,"For Some Venetians, This Overgrown Island Is P...",https://www.nytimes.com/2025/08/15/world/europ...
4,2025-08-15-18-33-16 +0000,nyt,Man Rescued From Behind California Waterfall A...,https://www.nytimes.com/2025/08/15/us/californ...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2342/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
2,trump,63
6,putin,29
166,ukraine,15
5,summit,13
4,alaska,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
59,2025-08-15-15-56-47 +0000,nyt,"How Trump’s Meeting With Putin Could Unfold, a...",https://www.nytimes.com/2025/08/15/us/politics...,187
90,2025-08-15-13-25-54 +0000,nypost,Putin’s foreign minister trolls Ukraine by wea...,https://nypost.com/2025/08/15/world-news/putin...,165
208,2025-08-15-01-00-00 +0000,wsj,"President Trump, by meeting Vladimir Putin on ...",https://www.wsj.com/politics/national-security...,164
172,2025-08-15-07-00-55 +0000,wapo,"How Putin, ex-KGB officer, will seek to sway T...",https://www.washingtonpost.com/world/2025/08/1...,140
194,2025-08-15-02-38-36 +0000,nypost,Anchorage prepares for Trump-Putin summit as s...,https://nypost.com/2025/08/14/us-news/anchorag...,133


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
59,187,2025-08-15-15-56-47 +0000,nyt,"How Trump’s Meeting With Putin Could Unfold, a...",https://www.nytimes.com/2025/08/15/us/politics...
181,60,2025-08-15-04-27-14 +0000,nypost,AG Pam Bondi declares DC ‘will not remain a sa...,https://nypost.com/2025/08/15/us-news/ag-pam-b...
242,58,2025-08-14-22-26-00 +0000,wsj,California Democrats are planning their own re...,https://www.wsj.com/politics/policy/gavin-news...
208,42,2025-08-15-01-00-00 +0000,wsj,"President Trump, by meeting Vladimir Putin on ...",https://www.wsj.com/politics/national-security...
89,41,2025-08-15-13-29-05 +0000,nypost,Explosion at Russian gunpowder factory kills 5...,https://nypost.com/2025/08/15/world-news/russi...
147,39,2025-08-15-09-30-00 +0000,wsj,Tariffs haven’t caused inflation to surge as m...,https://www.wsj.com/economy/trade/trump-tariff...
243,37,2025-08-14-22-22-35 +0000,nypost,The first-ever World Humanoid Robot Games kick...,https://nypost.com/2025/08/14/tech/the-first-e...
205,32,2025-08-15-01-11-32 +0000,latimes,An L.A. high school senior was walking his dog...,https://www.latimes.com/california/story/2025-...
40,30,2025-08-15-16-54-21 +0000,bbc,Washington DC sues federal government over pol...,https://www.bbc.com/news/articles/c2018769n1yo...
209,30,2025-08-15-01-00-00 +0000,wsj,The nine law firms that promised President Tru...,https://www.wsj.com/politics/policy/trump-pro-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
